# Examining the effects of ownership on software quality
## The Case Of Lucene

We want to replicate the [study](http://dl.acm.org/citation.cfm?doid=2025113.2025119 "Examining the effects of ownership on software quality") done by Bird et al. and published at FSE'11. The idea is to see the results of a similar investigation on an OSS system. We select [Lucene](https://lucene.apache.org/core/), a search engine written in Java.

## Data collection

First we need to get the data to create our **table**, in other words we do what is called *data collection*.

In our case, we are interested in checking the relation between some ownership related metrics and post-release bugs. We investigating this relation at *file level*, because we focus on Java and in this language the building blocks are the classes, which most of the time correspond 1-to-1 to files.

This means that our table will have one row per each source code file and as many columns as the metrics we want to compute for that file, plus one column with the number of post release bugs.

### Collecting git data

For computing most of the metrics we want to investigate (e.g., how many people changed a file in its entire history) we need to know the history of files. We can do so by analyzing the *versioning system*. In our case, Lucene has a Subversion repository, but a [git mirror](https://github.com/apache/lucene-solr.git) is also available. We use the git repository as it allows to have the entire history locally, thus making the computations faster.

We clone the repository. For this we use the python library 'sh'.

In [1]:
import sh

We start by cloning the repository

In [2]:
sh.git.clone("https://github.com/apache/lucene-solr.git")

And we make sure that we point our 'git' command to the right directory.

In [3]:
git = sh.git.bake(_cwd='lucene-solr')
git.status()

On branch master
Your branch is up-to-date with 'origin/master'.
nothing to commit, working directory clean

To perform the replication, we could either reason in terms of releases (see [list of Lucene releases](http://archive.apache.org/dist/lucene/java/)), or we could just inspect the 'trunk' in the versioning system and start from a given date.

In this assignment, we go for the second option: We consider the 'trunk' (main branch in svn) and focus on a 6-month period in which we look at the bugs occurring to the files existing at that moment. Concerning bug data, you will consider a time window from Feb 01, 2015 to Jul 31, 2015.

Let's retrieve the list of files existing in the trunk on Feb 01, 2015.

In [119]:
shaFeb15 = (git("rev-list","-n 1","--before=\"2015-02-01 00:01\"","master")).stdout[:-1]
shaFeb15

'b5db48c783e9d0dc19c087101f03d8834b201106'

In [120]:
git.checkout(shaFeb15)
git.status()

HEAD detached at b5db48c
nothing to commit, working directory clean

After getting the snapshot right, we will get all the java files inside the repository at a given snapshot. We do this using dict which has a key of full path and the corresponding package name and the filename.

need to check how to make a proper package name. i.e. how to extract org.apache.lucene from lucene/test-framework/src/java/org/apache/lucene/search/ShardSearchingTestBase.java

In [121]:
listfiles = git("ls-files").split("\n")

java_files = {}

for i in listfiles:
    if i[-5:] == ".java":
        split_file = i.split("/")
        pkg = '/'.join(split_file[:-1]) #need to checkout later
        filename = '/'.join(split_file[-1:]) 
        java_files[i] = [filename,pkg,0]

print java_files


{u'lucene/core/src/java/org/apache/lucene/codecs/PostingsReaderBase.java': [u'PostingsReaderBase.java', u'lucene/core/src/java/org/apache/lucene/codecs', 0], u'lucene/queryparser/src/java/org/apache/lucene/queryparser/flexible/core/nodes/OrQueryNode.java': [u'OrQueryNode.java', u'lucene/queryparser/src/java/org/apache/lucene/queryparser/flexible/core/nodes', 0], u'solr/solrj/src/java/org/apache/solr/client/solrj/response/FieldStatsInfo.java': [u'FieldStatsInfo.java', u'solr/solrj/src/java/org/apache/solr/client/solrj/response', 0], u'solr/contrib/dataimporthandler/src/java/org/apache/solr/handler/dataimport/DataImportHandler.java': [u'DataImportHandler.java', u'solr/contrib/dataimporthandler/src/java/org/apache/solr/handler/dataimport', 0], u'lucene/queryparser/src/java/org/apache/lucene/queryparser/flexible/standard/config/FieldBoostMapFCListener.java': [u'FieldBoostMapFCListener.java', u'lucene/queryparser/src/java/org/apache/lucene/queryparser/flexible/standard/config', 0], u'solr/s

Now I need to get the commit before Aug 01, 2015.

In [122]:
shaAug15 = (git("rev-list","-n 1","--before=\"2015-08-01 00:01\"","master")).stdout[:-1]
shaAug15

'7b412fdc630081ef8299952e1ea583eee5e89197'

In [123]:
git.checkout(shaAug15)
git.status()

HEAD detached at 7b412fd
nothing to commit, working directory clean

Now we need to get the bugs between 2015-02-01 00:02 until 2015-08-01 00:00. We will analyze the JIRA of lucene_solr project.

In [124]:
from jira import JIRA
jira = JIRA("https://issues.apache.org/jira/")
    
checkpoint = 100
total_issues = 0

issues = []

while checkpoint == 100:
    issues_page = jira.search_issues("(project=LUCENE or project=SOLR) and (created >= '2015-02-01 00:02' and created <='2015-08-01 00:00') and status = 'Closed'", startAt=total_issues, maxResults = 100)
    for i in issues_page:
        issues.append(i.key)
    checkpoint = len(issues_page)
    total_issues += checkpoint
    print total_issues

# up until this point, we already have a list of all bug's keys
# next we need to find this key in git log commit to find which file is affected by the corresponding bug
# let's make a function i.e. find_file(bug_key) 

100
200
300
400
500
600
700
733


In [125]:
print issues

[u'SOLR-7855', u'SOLR-7854', u'SOLR-7852', u'SOLR-7851', u'SOLR-7849', u'SOLR-7847', u'SOLR-7843', u'SOLR-7842', u'SOLR-7841', u'SOLR-7840', u'SOLR-7839', u'SOLR-7838', u'SOLR-7837', u'SOLR-7836', u'SOLR-7832', u'SOLR-7831', u'SOLR-7829', u'SOLR-7825', u'SOLR-7823', u'SOLR-7822', u'SOLR-7821', u'SOLR-7819', u'SOLR-7818', u'SOLR-7815', u'SOLR-7814', u'SOLR-7813', u'SOLR-7810', u'SOLR-7809', u'SOLR-7807', u'SOLR-7805', u'SOLR-7803', u'SOLR-7800', u'SOLR-7799', u'SOLR-7797', u'SOLR-7792', u'SOLR-7791', u'SOLR-7790', u'SOLR-7789', u'SOLR-7788', u'SOLR-7787', u'SOLR-7785', u'SOLR-7784', u'SOLR-7783', u'SOLR-7782', u'SOLR-7781', u'SOLR-7780', u'SOLR-7778', u'SOLR-7776', u'SOLR-7775', u'SOLR-7774', u'SOLR-7771', u'SOLR-7769', u'SOLR-7765', u'SOLR-7763', u'SOLR-7760', u'SOLR-7757', u'SOLR-7756', u'SOLR-7755', u'SOLR-7751', u'SOLR-7750', u'SOLR-7748', u'SOLR-7746', u'SOLR-7743', u'SOLR-7742', u'SOLR-7741', u'SOLR-7740', u'SOLR-7739', u'SOLR-7737', u'SOLR-7735', u'SOLR-7732', u'SOLR-7731', u'SOL

In [126]:
import re

gitlog = (git("log","--after=\"2015-02-01 00:00\"","--before=\"2015-08-01 00:00\"","--format=%H#%s")).split("\n")
commit_to_bug = []
for counter in range(0,len(gitlog)-1):
    i = gitlog[counter]
    text_split = i.split("#")
    commit_id = text_split[0]
    bug_key = re.findall(r"(\bSOLR\b-[0-9]+|\bLUCENE\b-[0-9]+)",text_split[1])
    if bug_key != [] and bug_key[0] in issues:
        commit_to_bug.append([commit_id, bug_key[0]])

print commit_to_bug

[[u'22d67a637acb75b486f4e6ff9f599f0f4a505c1a', u'SOLR-7823'], [u'f8ae631751ae98ca770d8f387793d9846db62c48', u'LUCENE-6702'], [u'caace607e8ae1b807f31048b13fdde44bd037e01', u'SOLR-7852'], [u'ba50839c99222d422d17ece938d40240b3fcff6e', u'SOLR-7851'], [u'a3cff55407a4478ddd9d82fd3ec6fa787175d513', u'LUCENE-6621'], [u'c75e5c61d7c4d6df6586eedd00948c2fe7e216eb', u'LUCENE-6707'], [u'c8bdf9bf19829fd37eb96d128bd71aff4de63cbb', u'SOLR-7840'], [u'a5d3bd5645e465df6389bcd7ad24ebf47fe0a2cd', u'SOLR-7842'], [u'65fee23d4769e63da92a2150021b53b3fb75baa2', u'LUCENE-6570'], [u'7023d92ca8e21911faa88d3638aea3fbbd6a4947', u'LUCENE-6334'], [u'5b0e8d1d41979137f19bf2b78706467177a602aa', u'LUCENE-6531'], [u'0155076b1f1b430f9d5e5eff3c0fef73dad48d5e', u'SOLR-7829'], [u'e1558fef1c1f02de488f13e5604152bc7529c778', u'LUCENE-6695'], [u'5d16b9739627a29c2d3e172cda55405e6d1d6303', u'LUCENE-6688'], [u'2d5f162bb8cd35095a127d458180a6af660c6631', u'SOLR-7735'], [u'8649f3f3bd7e9856e7732ca13dffd80fbae3695e', u'SOLR-7825'], [u'38c3

So, up until this point, we already have a mapping of commit_id and bugs inside the interval observed. Next step is to find the changed files for each commit and then compute the number of bugs for each file.

In [127]:
# git diff-tree --name-only -r commit_id
# for each files in all commit, find the changed files

java_files_copy = java_files

for i in commit_to_bug:
    commit_id_bug = i[0]
    changed_files = git("diff-tree","-r","--no-commit-id","--name-only",commit_id_bug)
    x = changed_files.split("\n")
    for y in x:
        if y in java_files:
            java_files_copy[y][2] += 1



In [128]:
#print java_files_copy

for key in java_files_copy:
    print key + "|" + java_files_copy[key][0] + "|" + java_files_copy[key][1] + "|" + str(java_files_copy[key][2])

lucene/core/src/java/org/apache/lucene/codecs/PostingsReaderBase.java|PostingsReaderBase.java|lucene/core/src/java/org/apache/lucene/codecs|1
lucene/queryparser/src/java/org/apache/lucene/queryparser/flexible/core/nodes/OrQueryNode.java|OrQueryNode.java|lucene/queryparser/src/java/org/apache/lucene/queryparser/flexible/core/nodes|0
solr/solrj/src/java/org/apache/solr/client/solrj/response/FieldStatsInfo.java|FieldStatsInfo.java|solr/solrj/src/java/org/apache/solr/client/solrj/response|0
solr/contrib/dataimporthandler/src/java/org/apache/solr/handler/dataimport/DataImportHandler.java|DataImportHandler.java|solr/contrib/dataimporthandler/src/java/org/apache/solr/handler/dataimport|1
lucene/queryparser/src/java/org/apache/lucene/queryparser/flexible/standard/config/FieldBoostMapFCListener.java|FieldBoostMapFCListener.java|lucene/queryparser/src/java/org/apache/lucene/queryparser/flexible/standard/config|0
solr/solrj/src/java/org/apache/solr/client/solrj/response/AnalysisResponseBase.java|